In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

df = pd.read_csv('preprocessed.csv')
df.head()

,Unnamed: 0,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_ACTIVE_CLOSED,CREDIT_ACTIVE_ACTIVE,CREDIT_ACTIVE_SOLD,CREDIT_ACTIVE_BAD_DEBT,CREDIT_DAY_OVERDUE,AVG_AMT_CREDIT_SUM,AMT_APPLICATION_APPROVED,AMT_APPLICATION_REFUSED,AMT_APPLICATION_CANCELED,AMT_APPLICATION_UNUSED,INCOME_LOAN_ANNUITY_PCT,AGE_BUCKET
0,0,1,Cash loans,M,N,Y,0,2.02,4.07,0.25,3.51,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,25,1,9.0,5,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,0.262949,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,2.0,0.0,0.0,0.0,108131.945625,1.0,0.0,0.0,0.0,12.38,20-30
1,1,0,Cash loans,F,N,N,0,2.70,12.94,0.36,11.29,Family,State servant,Higher education,Married,House / apartment,0.003541,45,3,3.0,0,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,0.622246,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,254350.125000,3.0,0.0,0.0,0.0,13.33,40-50
2,2,0,Revolving loans,M,Y,Y,0,0.68,1.35,0.07,1.35,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,52,0,11.0,6,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,0.555912,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,94518.900000,1.0,0.0,0.0,0.0,10.29,50-60
3,4,0,Cash loans,M,N,Y,0,1.22,5.13,0.22,5.13,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,54,8,11.0,9,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,0.322738,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,146250.000000,6.0,0.0,0.0,0.0,18.03,50-60
4,5,0,Cash loans,M,N,Y,0,0.99,4.90,0.28,4.54,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,0.035792,46,4,13.0,1,1,1,1,1,1,0,Laborers,2.0,2,2,WEDNESDAY,16,0,0,0,0,0,0,0.354225,0.0,0.0,0.0,0.0,-2536.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,156148.500000,4.0,0.0,1.0,0.0,28.28,40-50


In [24]:
df = df[df['AGE_BUCKET'].notna()]

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210842 entries, 0 to 210842
Data columns (total 83 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   210842 non-null  int64  
 1   TARGET                       210842 non-null  int64  
 2   NAME_CONTRACT_TYPE           210842 non-null  object 
 3   CODE_GENDER                  210842 non-null  object 
 4   FLAG_OWN_CAR                 210842 non-null  object 
 5   FLAG_OWN_REALTY              210842 non-null  object 
 6   CNT_CHILDREN                 210842 non-null  int64  
 7   AMT_INCOME_TOTAL             210842 non-null  float64
 8   AMT_CREDIT                   210842 non-null  float64
 9   AMT_ANNUITY                  210842 non-null  float64
 10  AMT_GOODS_PRICE              210842 non-null  float64
 11  NAME_TYPE_SUITE              210842 non-null  object 
 12  NAME_INCOME_TYPE             210842 non-null  object 
 13 

In [26]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
pd.set_option('display.max_colwidth', None)

In [27]:
from sklearn.preprocessing import LabelEncoder
def label_encoded(feat):
    le = LabelEncoder()
    le.fit(feat)
    print(feat.name,le.classes_)
    return le.transform(feat)

In [28]:
# encoding categorical variables in order to use it in future models
for col in df.select_dtypes(include=['object']).columns:
    df[str(col)] = label_encoded(df[str(col)])

NAME_CONTRACT_TYPE ['Cash loans' 'Revolving loans']
CODE_GENDER ['F' 'M']
FLAG_OWN_CAR ['N' 'Y']
FLAG_OWN_REALTY ['N' 'Y']
NAME_TYPE_SUITE ['Children' 'Family' 'Group of people' 'Other_A' 'Other_B'
 'Spouse, partner' 'Unaccompanied']
NAME_INCOME_TYPE ['Commercial associate' 'Pensioner' 'State servant' 'Student' 'Unemployed'
 'Working']
NAME_EDUCATION_TYPE ['Academic degree' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Secondary / secondary special']
NAME_FAMILY_STATUS ['Civil marriage' 'Married' 'Separated' 'Single / not married' 'Widow']
NAME_HOUSING_TYPE ['Co-op apartment' 'House / apartment' 'Municipal apartment'
 'Office apartment' 'Rented apartment' 'With parents']
OCCUPATION_TYPE ['Accountants' 'Cleaning staff' 'Cooking staff' 'Core staff' 'Drivers'
 'HR staff' 'High skill tech staff' 'IT staff' 'Laborers'
 'Low-skill Laborers' 'Managers' 'Medicine staff' 'Private service staff'
 'Realty agents' 'Sales staff' 'Secretaries' 'Security staff'
 'Waiters/barmen staff' 'u

In [29]:
X = df.drop(['TARGET'],axis=1).values
y = df['TARGET'].values

In [30]:
zero = df[df['TARGET'] == 0].head(16715)
one = df[df['TARGET'] == 1]
half = pd.concat([zero, one])

X_bal = half.drop(['TARGET'],axis=1).values
y_bal = half['TARGET'].values

In [31]:
len(df[df['TARGET'] == 1]['TARGET'])

16715

In [32]:
X.shape

(210842, 82)

In [33]:
y.shape

(210842,)

In [34]:
X_bal.shape

(33430, 82)

In [35]:
y_bal.shape

(33430,)

In [36]:
#LDA
le = LabelEncoder()
y = le.fit_transform(df['NAME_CONTRACT_TYPE'])

In [37]:
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
y_pred = lda.fit(X, y)
X_lda = lda.transform(X)

In [38]:
#Define method to evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#evaluate model
scores = cross_val_score(lda, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores))   

0.9918280042201194


In [39]:
#QDA
qda = QuadraticDiscriminantAnalysis()
X_qda = qda.fit(X, y)

C:\Users\anast\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\discriminant_analysis.py:887: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [40]:
#Define method to evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#evaluate model
scores = cross_val_score(qda, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores)) 

1.0
